In [48]:
# Step 1: Import Libraries
import numpy as np
from sklearn.linear_model import OrthogonalMatchingPursuit
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt


In [49]:

# Step 2: Create a Synthetic Dataset
# Set random seed for reproducibility
np.random.seed(0)

# Number of samples per class
n_samples_per_class = 50

# Generate synthetic data for three classes
class_1 = np.random.randn(n_samples_per_class, 2) +  np.array([2, 2])
class_2 = np.random.randn(n_samples_per_class, 2) + np.array([-2, -2])
class_3 = np.random.randn(n_samples_per_class, 2) + np.array([-2, 2])

# print(f'{class_1}\n')
# print(f'{class_2}\n')
# print(f'{class_3}\n')

# Combine the data
X = np.vstack((class_1, class_2, class_3))
# Class/true label
y = np.array([0]*n_samples_per_class + [1]*n_samples_per_class + [2]*n_samples_per_class)

# print(f'Actual Label for these 150 samples {y}\n')

# Plot the data
# plt.scatter(class_1[:, 0], class_1[:, 1], label='Class 0')
# plt.scatter(class_2[:, 0], class_2[:, 1], label='Class 1')
# plt.scatter(class_3[:, 0], class_3[:, 1], label='Class 2')
# plt.legend()
# plt.title('Synthetic Dataset')
# plt.show()



In [50]:
# Split the data into training and testing sets
X_train, X_test, y_label_train, y_label_test = train_test_split(X, y, test_size=0.2, stratify=y)

# Normalize the data
X_train = normalize(X_train)
X_test = normalize(X_test)


In [51]:
# Create dictionaries for each class
D_classes = {}
classes = np.unique(y_label_train)

for cls in classes:
    D_classes[cls] = X_train[y_label_train == cls].T  # Transpose for computation, the 'true' index will form an array
 
# D = np.hstack([D_classes[cls] for cls in classes])  # Concatenate the dictionaries
D = np.hstack([D_classes[0], D_classes[1], D_classes[2]])
    

In [52]:
# Sparse Coding Using Orthogonal Matching Pursuit (OMP) - We'll use OMP to find the sparse coefficients.
# Initialize the OMP model
omp = OrthogonalMatchingPursuit(n_nonzero_coefs=10)

# List to store predicted classes
y_pred = []

print(f'{D_classes}\n')
# For each test sample
for i in range(X_test.shape[0]):
    y_i = X_test[i]
    
    # Fit the model to find sparse coefficients
    # [Fits the OMP model using the dictionary D and the test sample y_i. The goal is to find a sparse set of coefficients to represent y_i using D]
    omp.fit(D, y_i)

    # Get the sparse coefficient vector
    # [Gets the sparse coefficient vector found by OMP. This vector tells how much of each atom (column) in D is used to represent y_i.]
    x_sparse = omp.coef_

    # Reconstruct the test sample using class-specific dictionaries
    residuals = []
    start = 0
    for cls in classes:
        D_cls = D_classes[cls]
        n_atoms = D_cls.shape[1]
        x_cls = x_sparse[start:start + n_atoms]
        y_reconstructed = D_cls @ x_cls

        # Calculate the residual (reconstruction error)
        residual = np.linalg.norm(y_i - y_reconstructed)
        residuals.append(residual)

        start += n_atoms

    # Assign the class with the smallest residual
    y_pred.append(np.argmin(residuals))


{0: array([[ 0.63036116,  0.6183736 ,  0.29105602,  0.27657134,  0.71348199,
         0.69033486,  0.71016817,  0.85386341,  0.66794488,  0.66293669,
         0.21115913,  0.87613872,  0.63461699,  0.74843225,  0.25978505,
         0.47002912,  0.78846135,  0.67147454,  0.44400763,  0.62209912,
         0.8895065 ,  0.72322313,  0.79292925,  0.41996262,  0.89606826,
         0.57477123,  0.07414275,  0.86781596,  0.60817346,  0.87715443,
         0.68871821,  0.91561136,  0.62907191,  0.84737321,  0.46901992,
         0.66705573,  0.84316893,  0.52736495,  0.96676971, -0.20400821],
       [ 0.77630201,  0.78588427,  0.95670601,  0.96099339,  0.70067357,
         0.72349001,  0.70403208,  0.52049714,  0.74421075,  0.74867546,
         0.9774517 ,  0.48205906,  0.77282681,  0.66321125,  0.96566647,
         0.88265091,  0.6150843 ,  0.74102763,  0.89602301,  0.78293849,
         0.45692251,  0.69061444,  0.60931372,  0.9075414 ,  0.44391629,
         0.81831414,  0.99724764,  0.49688575,

/home/arefin/anaconda3/envs/CRKG/lib/python3.12/site-packages/sklearn/linear_model/_omp.py:445: RuntimeWarning: Orthogonal matching pursuit ended prematurely due to linear dependence in the dictionary. The requested precision might not have been met.
  out = _cholesky_omp(
/home/arefin/anaconda3/envs/CRKG/lib/python3.12/site-packages/sklearn/linear_model/_omp.py:445: RuntimeWarning: Orthogonal matching pursuit ended prematurely due to linear dependence in the dictionary. The requested precision might not have been met.
  out = _cholesky_omp(
/home/arefin/anaconda3/envs/CRKG/lib/python3.12/site-packages/sklearn/linear_model/_omp.py:445: RuntimeWarning: Orthogonal matching pursuit ended prematurely due to linear dependence in the dictionary. The requested precision might not have been met.
  out = _cholesky_omp(
/home/arefin/anaconda3/envs/CRKG/lib/python3.12/site-packages/sklearn/linear_model/_omp.py:445: RuntimeWarning: Orthogonal matching pursuit ended prematurely due to linear depend

In [53]:
import numpy as np
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.decomposition import DictionaryLearning
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Step 1: Load the digits dataset
digits = load_digits()
X, y = digits.data, digits.target  # Features and labels

# Step 2: Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Step 3: Dictionary learning (learning sparse representations)
# Let's learn a dictionary with 64 components (same as the number of pixels per image)
dictionary = DictionaryLearning(n_components=64, transform_algorithm='lasso_lars', random_state=42)
X_train_sparse = dictionary.fit_transform(X_train)

# Step 4: Transform the test data into sparse representations
X_test_sparse = dictionary.transform(X_test)

# Step 5: Train a classifier on the sparse representations
clf = LogisticRegression(max_iter=1000, random_state=42)
clf.fit(X_train_sparse, y_train)

# Step 6: Predict and evaluate on test data
y_pred = clf.predict(X_test_sparse)
accuracy = accuracy_score(y_test, y_pred)

print(f"Classification accuracy using SRC: {accuracy * 100:.2f}%")


KeyboardInterrupt: 